In [27]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# Configuration
config = {
    'trade_date': '2024-03-01',
    'stock_name': 'NIFTY',
    'entry_time': '09:17:59',
    'target_multiplier': 1.2,
    'stoploss_multiplier': 0.2,
    'report_name': 'BUYING_V1_{YYYYMMDD}.xlsx',
    'start_date': '2024-03-01',
    'end_date': '2024-03-10'
}

In [28]:
# Database credentials
DB_USER = "root"
DB_PASS = quote_plus(r"Naman@123")
DB_HOST = "localhost"
DB_PORT = 3306
DB_NAME = "fnodata2024"

In [29]:
engine = create_engine(f"mysql+mysqlconnector://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")


In [30]:
# Function to fetch data from a table
def fetch_data(table_name):
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, con=engine)
    return df

# Fetch data from the database
spot_data = fetch_data("spot_data")
options_data = fetch_data("fno_nifty")

In [31]:
spot_data = spot_data[(spot_data['tr_time'] >= '09:15:00') & (spot_data['tr_time'] <= '15:30:00')] # type: ignore
options_data = options_data[(options_data['tr_time'] >= '09:15:00') & (options_data['tr_time'] <= '15:30:00')] # type: ignore

In [32]:
def check_exit_conditions(row, exit_ce, exit_pe, sl_ce, tg_ce, sl_pe, tg_pe):
    current_time = row['tr_time']
    
    if row['otype'] == 'CE' and not exit_ce['time']:
        if row['tr_close'] <= sl_ce:
            exit_ce.update({'time': current_time, 'price': row['tr_close'], 'type': 'SL'})
        elif row['tr_close'] >= tg_ce:
            exit_ce.update({'time': current_time, 'price': row['tr_close'], 'type': 'TG'})
    
    if row['otype'] == 'PE' and not exit_pe['time']:
        if row['tr_close'] <= sl_pe:
            exit_pe.update({'time': current_time, 'price': row['tr_close'], 'type': 'SL'})
        elif row['tr_close'] >= tg_pe:
            exit_pe.update({'time': current_time, 'price': row['tr_close'], 'type': 'TG'})
    
    return exit_ce, exit_pe

def trade_one_day(trade_date, config, spot_data, options_data):
    entry_time = config['entry_time']
    stoploss_multiplier = config['stoploss_multiplier']
    target_multiplier = config['target_multiplier']
    
    def get_spot_and_strike_price(trade_date, spot_data):
        trade_data = spot_data[spot_data['tr_date'] == trade_date]
        if trade_data.empty:
            return None, None
        spot_price = trade_data.iloc[0]['tr_close']
        strike_price = round(spot_price / 50) * 50
        return spot_price, strike_price

    def calculate_prices(entry_price, stoploss_multiplier, target_multiplier):
        stoploss_price = entry_price * (1 - stoploss_multiplier)
        target_price = entry_price * (1 + target_multiplier)
        return stoploss_price, target_price
    
    spot_price, strike_price = get_spot_and_strike_price(trade_date, spot_data)
    if spot_price is None:
        print(f"No data for trade date: {trade_date}")
        return None
    
    entry_data = options_data[(options_data['tr_date'] == trade_date) & (options_data['tr_time'] == entry_time)]
    if entry_data.empty:
        print(f"No entry data for trade date: {trade_date} and entry time: {entry_time}")
        return None

    try:
        entry_price_ce = entry_data[entry_data['otype'] == 'CE']['tr_close'].values[0]
        entry_price_pe = entry_data[entry_data['otype'] == 'PE']['tr_close'].values[0]
    except IndexError:
        print(f"Entry data is missing for CE or PE on {trade_date} at {entry_time}")
        return None

    sl_ce, tg_ce = calculate_prices(entry_price_ce, stoploss_multiplier, target_multiplier)
    sl_pe, tg_pe = calculate_prices(entry_price_pe, stoploss_multiplier, target_multiplier)
    
    exit_ce = {'time': None, 'price': None, 'type': None, 'target': tg_ce, 'stoploss': sl_ce}
    exit_pe = {'time': None, 'price': None, 'type': None, 'target': tg_pe, 'stoploss': sl_pe}
    
    trade_data = options_data[options_data['tr_date'] == trade_date]
    
    for _, row in trade_data.iterrows():
        exit_ce, exit_pe = check_exit_conditions(row, exit_ce, exit_pe, sl_ce, tg_ce, sl_pe, tg_pe)
        if exit_ce['time'] and exit_pe['time']:
            break

    return {
        'Entry_Date': trade_date,
        'Stock_Name': config['stock_name'],
        'Entry_Time': entry_time,
        'Strike_Price': strike_price,
        'CE_Entry_Price': entry_price_ce,
        'CE_Exit_Time': exit_ce['time'],
        'CE_Exit_Price': exit_ce['price'],
        'CE_Exit_Type': exit_ce['type'],
        'CE_Target_Price': exit_ce['target'],
        'CE_Stoploss_Price': exit_ce['stoploss'],
        'PE_Entry_Price': entry_price_pe,
        'PE_Exit_Time': exit_pe['time'],
        'PE_Exit_Price': exit_pe['price'],
        'PE_Exit_Type': exit_pe['type'],
        'PE_Target_Price': exit_pe['target'],
        'PE_Stoploss_Price': exit_pe['stoploss']
    }

In [33]:
def check_exit_conditions(row, exit_ce, exit_pe, sl_ce, tg_ce, sl_pe, tg_pe):
    current_time = row['tr_time']
    
    if row['otype'] == 'CE' and not exit_ce['time']:
        if row['tr_close'] <= sl_ce:
            exit_ce.update({'time': current_time, 'price': row['tr_close'], 'type': 'SL'})
        elif row['tr_close'] >= tg_ce:
            exit_ce.update({'time': current_time, 'price': row['tr_close'], 'type': 'TG'})
    
    if row['otype'] == 'PE' and not exit_pe['time']:
        if row['tr_close'] <= sl_pe:
            exit_pe.update({'time': current_time, 'price': row['tr_close'], 'type': 'SL'})
        elif row['tr_close'] >= tg_pe:
            exit_pe.update({'time': current_time, 'price': row['tr_close'], 'type': 'TG'})
    
    return exit_ce, exit_pe

def trade_one_day(trade_date, config, spot_data, options_data):
    entry_time = config['entry_time']
    stoploss_multiplier = config['stoploss_multiplier']
    target_multiplier = config['target_multiplier']
    
    def get_spot_and_strike_price(trade_date, spot_data):
        trade_data = spot_data[spot_data['tr_date'] == trade_date]
        if trade_data.empty:
            return None, None
        spot_price = trade_data.iloc[0]['tr_close']
        strike_price = round(spot_price / 50) * 50
        return spot_price, strike_price

    def calculate_prices(entry_price, stoploss_multiplier, target_multiplier):
        stoploss_price = entry_price * (1 - stoploss_multiplier)
        target_price = entry_price * (1 + target_multiplier)
        return stoploss_price, target_price
    
    spot_price, strike_price = get_spot_and_strike_price(trade_date, spot_data)
    if spot_price is None:
        print(f"No data for trade date: {trade_date}")
        return None
    
    entry_data = options_data[(options_data['tr_date'] == trade_date) & (options_data['tr_time'] == entry_time)]
    if entry_data.empty:
        print(f"No entry data for trade date: {trade_date} and entry time: {entry_time}")
        return None

    try:
        entry_price_ce = entry_data[entry_data['otype'] == 'CE']['tr_close'].values[0]
        entry_price_pe = entry_data[entry_data['otype'] == 'PE']['tr_close'].values[0]
    except IndexError:
        print(f"Entry data is missing for CE or PE on {trade_date} at {entry_time}")
        return None

    sl_ce, tg_ce = calculate_prices(entry_price_ce, stoploss_multiplier, target_multiplier)
    sl_pe, tg_pe = calculate_prices(entry_price_pe, stoploss_multiplier, target_multiplier)
    
    exit_ce = {'time': None, 'price': None, 'type': None, 'target': tg_ce, 'stoploss': sl_ce}
    exit_pe = {'time': None, 'price': None, 'type': None, 'target': tg_pe, 'stoploss': sl_pe}
    
    trade_data = options_data[options_data['tr_date'] == trade_date]
    
    for _, row in trade_data.iterrows():
        exit_ce, exit_pe = check_exit_conditions(row, exit_ce, exit_pe, sl_ce, tg_ce, sl_pe, tg_pe)
        if exit_ce['time'] and exit_pe['time']:
            break

    return {
        'Entry_Date': trade_date,
        'Stock_Name': config['stock_name'],
        'Entry_Time': entry_time,
        'Strike_Price': strike_price,
        'CE_Entry_Price': entry_price_ce,
        'CE_Exit_Time': exit_ce['time'],
        'CE_Exit_Price': exit_ce['price'],
        'CE_Exit_Type': exit_ce['type'],
        'CE_Target_Price': exit_ce['target'],
        'CE_Stoploss_Price': exit_ce['stoploss'],
        'PE_Entry_Price': entry_price_pe,
        'PE_Exit_Time': exit_pe['time'],
        'PE_Exit_Price': exit_pe['price'],
        'PE_Exit_Type': exit_pe['type'],
        'PE_Target_Price': exit_pe['target'],
        'PE_Stoploss_Price': exit_pe['stoploss']
    }

In [34]:
def collect_trade_results(config, spot_data, options_data):
  start_date = datetime.strptime(config['start_date'], '%Y-%m-%d')
  end_date = datetime.strptime(config['end_date'], '%Y-%m-%d')

  def date_range(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
      yield start_date + timedelta(n)

  all_results = [
      trade_one_day(single_date.strftime('%Y-%m-%d'), config, spot_data, options_data)
      for single_date in date_range(start_date, end_date)
      if trade_one_day(single_date.strftime('%Y-%m-%d'), config, spot_data, options_data) is not None
  ]

  # Create DataFrame from results
  df = pd.DataFrame(all_results)
  print(df.to_string())

  return df  # You can also return the DataFrame if needed

# Collect trade results and store in a DataFrame
results_df = collect_trade_results(config, spot_data, options_data)

No data for trade date: 2024-03-02
No data for trade date: 2024-03-03
No data for trade date: 2024-03-08
No data for trade date: 2024-03-09
No data for trade date: 2024-03-10
   Entry_Date Stock_Name Entry_Time  Strike_Price  CE_Entry_Price CE_Exit_Time  CE_Exit_Price CE_Exit_Type  CE_Target_Price  CE_Stoploss_Price  PE_Entry_Price PE_Exit_Time  PE_Exit_Price PE_Exit_Type  PE_Target_Price  PE_Stoploss_Price
0  2024-03-01      NIFTY   09:17:59         22100          1655.0     09:15:59           3.05           SL          3641.00            1324.00            1.55     09:15:59          267.8           TG             3.41               1.24
1  2024-03-04      NIFTY   09:17:59         22400             0.9     09:15:59           2.05           TG             1.98               0.72            3.20     09:15:59            1.2           SL             7.04               2.56
2  2024-03-05      NIFTY   09:17:59         22350           330.0     09:15:59        1350.85           TG           